<h1 style="background-color:#239B56;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🗺️ הסבת שכבת תשריטי תיעוד גבולות בשטח לא מוסדר לבנק"ל מודרני 🗺️

In [ ]:
print('9. Unregistered Tatag borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  9.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבה מקומית של גבולות תת"ג בשטח לא מוסדר

In [ ]:
unreg_tatag_borders = gpd.read_file(CNFG.Local + 'UnregisterdTatag.gdb', layer = 'UnregisterdTatagBorders', include_fields = ['DocName', 'geometry']).set_crs(2039)

<h1 style="text-align:right;font-size:100%">
טבלה מקומית של גבולות תת"ג בשטח לא מוסדר

In [ ]:
unreg_tatag_data_cols = ['DocName', 'SURVEYOR', 'GUSH_NUM']

unreg_tatag_data = gpd.read_file(CNFG.Local + 'UnregisterdTatag.gdb', layer = 'UnregisterdTatagData', include_fields = unreg_tatag_data_cols, ignore_geometry = True)

del unreg_tatag_data_cols

<h1 style="text-align:right;font-size:100%">
טבלת מספרי זיהוי של גושים

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['BlockNumber', 'GlobalID', 'IsTax'])
Blocks = Blocks[Blocks['IsTax'] == 0].drop(columns = 'IsTax')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  9.b Cleaning data')

<h1 style="text-align:right;font-size:100%">
:שכבת הגבולות
<h1 style="text-align:right;font-size:100%">
יצירת שדה של שם התהליך (מספר מלבן)-

In [ ]:
unreg_tatag_borders[['ProcessName', 'dwg']] = unreg_tatag_borders['DocName'].str.split('.', expand=True)

unreg_tatag_borders['ProcessName'] = unreg_tatag_borders['ProcessName'].str.replace('-', '/')

unreg_tatag_borders.drop(columns=['DocName', 'dwg'], inplace=True)

<h1 style="text-align:right;font-size:100%">
:טבלת הגבולות
<h1 style="text-align:right;font-size:100%">
יצירת שדה של שם התהליך (מספר מלבן)-
<h1 style="text-align:right;font-size:100%">
קיבוץ נתוני הטבלה לפי שם התהליך-

In [ ]:
unreg_tatag_data[['ProcessName', 'dwg']] = unreg_tatag_data['DocName'].str.split('.', expand=True)

unreg_tatag_data['ProcessName'] = unreg_tatag_data['ProcessName'].str.replace('-', '/')

unreg_tatag_data.drop(columns=['DocName', 'dwg'], inplace=True)

unreg_tatag_data = unreg_tatag_data.groupby(by='ProcessName', as_index=False).max()

In [ ]:
print('  9.c Data conversion')

In [ ]:
unreg_tatag = unreg_tatag_borders.merge(unreg_tatag_data, on = 'ProcessName', how = 'inner')
del [unreg_tatag_borders, unreg_tatag_data]

<h1 style="text-align:right;font-size:100%">
מיזוג נתונים לשכבה אחת

<h1 style="text-align:right;font-size:100%">
מיזוג עם שכבת גושי קדסטר מודרניים לצורך קבלת מספר מזהה של גוש

In [ ]:
Blocks.rename(columns = {'GlobalID':'BlockUniqueID'}, inplace = True)

unreg_tatag['BlockUniqueID'] = unreg_tatag.merge(Blocks, left_on = 'GUSH_NUM', right_on = 'BlockNumber', how = 'left')['BlockUniqueID']
unreg_tatag.drop(columns='GUSH_NUM', inplace=True)

In [ ]:
print('  9.d Defining new layer')

In [ ]:
new_columns = np.dtype([
                        ("geometry", gpd.array.GeometryDtype),       
                       #("GlobalID", str),
                        ("ProcessName", str),
                        ("ProcessType", int),
                        ("Status", int),
                        ("BlockUniqueID", str),
                        ("GeodeticNetwork", int),
                        ("SurveyorLicenseID", int),
                        ("DataSource", int),
                       #("PlanName", str),  
                       #("ORACLE_ID", int),
                       #("Shape_Length", float),
                       #("Shape_Area", float),
                        ])

unreg_tatag_new = gpd.GeoDataFrame(data = np.empty(0, dtype = new_columns),  geometry = 'geometry', crs = 2039)

In [ ]:
unreg_tatag_new['geometry'] = unreg_tatag['geometry']

unreg_tatag_new['BlockUniqueID'] = unreg_tatag['BlockUniqueID']

unreg_tatag_new['ProcessType'] = 15        # תת"ג להסדר מקרקעין

unreg_tatag_new['ProcessName'] = unreg_tatag['ProcessName']

unreg_tatag_new['GeodeticNetwork'] = 3     # רשת תקפה

#unreg_tatag_new['Status'] = None          # לא קיימים סטאטוסים בנתונים הקיימים

unreg_tatag_new['SurveyorLicenseID'] = unreg_tatag['SURVEYOR'].astype(int)

unreg_tatag_new['DataSource'] = 5          # CAD

#unreg_tatag_new['PlanName'] = None

In [ ]:
unreg_tatag_new.sort_values('ProcessName', inplace = True)
del unreg_tatag

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

In [ ]:
print('  9.c Exporting results')

In [ ]:
unreg_tatag_new.to_file(CNFG.Results, layer = 'PT15', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')